In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [2]:
data = pd.read_csv('./data/event_transactions_CB.csv')
data.sample(10)

,event_id,attendee_id,order_id,purchase_quad,purchase_type,spend_cat,age_cat,gender_cat,attendee_frequency,attendee_zip,Latitude,Longitude
3729772,48,7367,10053,2,0,2,0,0,1,11691,40.600020,-73.759620
547684,23,388345,3860,4,1,0,3,1,1,18623,41.653217,-76.148700
2041913,36,48576,44023,1,0,0,0,0,1,98531,46.728383,-122.951420
4949399,9,359650,3404,2,1,2,3,1,2,61018,42.403305,-89.554660
1318768,30,42677,26708,1,0,1,0,1,1,76034,32.885062,-97.149230
1084028,28,46921,17195,1,1,1,0,0,2,98382,48.079979,-123.099160
4934400,9,17943,17071,2,0,2,0,1,1,98237,48.497601,-121.700180
759551,25,38239,9668,3,0,1,0,0,1,81138,37.197731,-105.429395
1061136,28,324322,24723,3,0,0,2,0,1,93610,37.100417,-120.278640
927500,27,38019,36204,2,1,2,0,0,2,54061,44.393405,-88.725880


In [3]:
data.shape

(4955031, 12)

In [4]:
data.columns

Index(['event_id', 'attendee_id', 'order_id', 'purchase_quad', 'purchase_type',
       'spend_cat', 'age_cat', 'gender_cat', 'attendee_frequency',
       'attendee_zip', 'Latitude', 'Longitude'],
      dtype='object')

In [23]:
def value_counts_sorted(series):
    vc = series.value_counts().sort_values(ascending = False)
    n = len(vc)
    return n, vc.index, vc.values    

In [24]:
n, ids, vals = value_counts_sorted(data.event_id)

In [25]:
n

53

In [26]:
data.event_id.value_counts().sort_values(ascending = False)

54    273573
39    273573
49    159528
37    159528
34    159528
36    159528
51    159528
52    159528
53    153600
44    153600
38    153600
29    153600
33    148524
50    148524
48    148524
35    148524
47    132678
32    132678
20    130845
27    127101
42    127101
19    109923
40     99318
31     99318
25     99318
46     99318
23     99063
26     77076
28     77076
30     77076
41     77076
43     77076
45     77076
24     76563
22     76563
10     54321
8      51357
9      49986
1      22755
18     21567
7      20328
16     19623
17     19101
12     17334
5      16149
11     14082
15     11841
2      11841
14     10662
21      9435
4       7242
13      6651
3       5703
Name: event_id, dtype: int64

In [27]:
vals

array([273573, 273573, 159528, 159528, 159528, 159528, 159528, 159528,
       153600, 153600, 153600, 153600, 148524, 148524, 148524, 148524,
       132678, 132678, 130845, 127101, 127101, 109923,  99318,  99318,
        99318,  99318,  99063,  77076,  77076,  77076,  77076,  77076,
        77076,  76563,  76563,  54321,  51357,  49986,  22755,  21567,
        20328,  19623,  19101,  17334,  16149,  14082,  11841,  11841,
        10662,   9435,   7242,   6651,   5703])